In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("traffic.csv")

In [3]:
proto_counts = df['Protocol'].value_counts()
print(proto_counts)

UDP     3598
SIP      596
HTTP     110
Name: Protocol, dtype: int64


In [4]:
df

,Packet_Num,Time,Source,Destination,Protocol,Length
0,147,33.320161,192.168.20.2,192.168.3.101,UDP,64
1,178,33.952039,192.168.20.2,192.168.3.101,UDP,64
2,186,34.208800,192.168.3.101,192.168.20.2,UDP,112
3,187,34.211274,192.168.3.101,192.168.20.2,UDP,300
4,188,34.213375,192.168.3.101,192.168.20.2,UDP,1150
...,...,...,...,...,...,...
4299,5851,50.932083,192.168.20.2,192.168.3.101,UDP,92
4300,5852,50.933567,192.168.3.101,192.168.20.2,UDP,1180
4301,5855,50.979146,192.168.3.101,192.168.20.2,UDP,1159
4302,5856,50.979177,192.168.3.101,192.168.20.2,UDP,1159


In [5]:
new_df = pd.DataFrame(columns=['Time','UE','App'])

for i, row in df.iterrows():
    
    
    if '192.168.3.101' in row['Source'] or '192.168.3.101' in row['Destination']:
        app = 'web-rtc'
    
    if '192.168.3.102' in row['Source'] or '192.168.3.102' in row['Destination']:
        app = 'sipp'
        
    if '192.168.3.103' in row['Source'] or '192.168.3.103' in row['Destination']:
        app = 'web-server'
        
    if '192.168.20.2' in row['Source'] or '192.168.20.2' in row['Destination']:
        ue = '1'
        
    if '192.168.20.3' in row['Source'] or '192.168.20.3' in row['Destination']:
        ue = '2'
        
    if '192.168.20.4' in row['Source'] or '192.168.20.4' in row['Destination']:
        ue = '3'
       
    data = {'Time': [row['Time']], 'UE':[ue], 'App':[app]}
    temp_df = pd.DataFrame.from_dict(data)
    
    
    new_df = new_df.append(temp_df, ignore_index=True)

In [6]:
new_df.loc[ new_df['App'] =='sipp']

,Time,UE,App
32,34.741746,2,sipp
33,34.771299,2,sipp
34,34.771317,2,sipp
35,34.771612,2,sipp
36,34.833435,2,sipp
...,...,...,...
3858,49.468581,2,sipp
3878,49.541626,2,sipp
3891,49.571315,2,sipp
3892,49.571334,2,sipp


In [7]:
new_df.to_csv("ue-app_time.csv",index=False)